<a href="https://colab.research.google.com/github/10ac-group10/Redash_Chatbot_LLM/blob/database/notebooks/data_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import pandas as pd
import os

In [117]:
# prompt: fetch data from google drive
def mount_drive():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

## Custom reusable functions 

In [118]:
# fetch data from google drive reusable function
def fetch_data(data_folder_name, file_name, google_colab=False):
    if google_colab:
        mount_drive()
        # Fetch data from google drive
        df = pd.read_csv(f"/content/drive/My Drive/10Academy/week3/data/{data_folder_name}/{file_name}.csv")
    else:
        # Fetch data from local
        df = pd.read_csv(f"../data/{data_folder_name}/{file_name}.csv")
    return df

In [119]:
def fetch_chart_data(data_folder_name):
  return fetch_data(data_folder_name, 'Chart data')

### Reusable code to generate dataframes and display unique values in teh csv files

In [120]:
def generate_dataframes(value_columns):
    dataframes = {}
    for name in value_columns.keys():
        dataframes[name] = fetch_chart_data(name)
    return dataframes

###### TODO - The subscription source needs further work, i'll ignore it for now

In [121]:
value_columns = {'Cities': 'Views', 'Content type': 'Views', 'Device type': 'Views', 'Geography': 'Views', 'New and returning viewers': 'Views', 'Operating system': 'Views', 'Sharing service': 'Shares'}


In [122]:
dataframes = generate_dataframes(value_columns)

# Conclusion from the results of the Device type, Cities and Content type data
-  We have successfully reshaped the data from long format to wide format, grouping by 'Device type', 'City name' and 'Content type' and 'Date' in this case
- We have also saved the reshaped data to a csv file
- The next step is to explore the other csv files and repeat the same process
- We will also explore the 'Geography', 'New and returning viewers', 'Operating system', 'Sharing service' and 'Subscription source' csv files

# Next steps:
- We need to have a reusable function to reshape the data and save the reshaped data to a csv file and just pass in the categorical column name, the dataframes and the names of the csv files

In [123]:
def save_preprocessed_data(df: pd.DataFrame, file_path: str) -> None:
    # Extract directory from file path
    dir_path = os.path.dirname(file_path)

    # Create the directory if it does not exist
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    # Save the DataFrame to a CSV file
    df.to_csv(file_path, index=False)

In [124]:
def reshape_data(dataframes, value_columns):
    reshaped_dataframes = {}
    for name, value_column in value_columns.items():
        # Pivot the data
        pivot_df = dataframes[name].pivot(index='Date', columns=name, values=value_column)

        # Reset the index
        pivot_df.reset_index(inplace=True)

        reshaped_dataframes[name] = pivot_df

    return reshaped_dataframes

def save_reshaped_data(reshaped_dataframes, value_columns):
    for name in value_columns.keys():
        # Save the reshaped data to a csv file
        save_preprocessed_data(reshaped_dataframes[name], f'../data/{name}/clean/Chart data.csv')

In [125]:
def preprocess_data(value_columns):
    dataframes = generate_dataframes(value_columns)
    reshaped_dataframes = reshape_data(dataframes, value_columns)
    save_reshaped_data(reshaped_dataframes, value_columns)

### Save the reshaped data to a csv file (save preprocessed data)

In [126]:
preprocess_data(value_columns)